In [1]:
import  pandas as pd
import  numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import xlrd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import openpyxl
import xgboost as xgb
from sklearn.feature_selection import RFE
from concurrent.futures import ThreadPoolExecutor
import queue
import warnings
import joblib
%run myfun.ipynb

warnings.filterwarnings("ignore")
clf = joblib.load("D:\\path\\model.m")


In [3]:
feature_start = 10
feature_num = 50


# result_path = 'D:\\LYC\exp_data\\0108\\random_forest_100\\f1f203_t1train_t3score'
result_path = 'D:\\result\\res'


#需要rescore的文件
path = 'D:\\result\\out_all_prsm.csv'

data=read_single_csv(path)
data['feature21'] = data['feature21'].transform(lambda x: x.fillna(x.mean()))
data['feature22'] = data['feature22'].transform(lambda x: x.fillna(x.mean()))
data['feature23'] = data['feature23'].transform(lambda x: x.fillna(x.mean()))
data['feature46'] = data['feature46'].fillna(0)
data['feature47'] = data['feature47'].fillna(0)
data['feature48'] = data['feature48'].fillna(0)
data['feature48'] = data['feature48'].replace([np.inf, -np.inf], 0)
data = data.dropna()
data = data.reset_index(drop=True)
test_feature = data[list(data.columns)[feature_start:feature_start+feature_num]]


probabilities = clf.predict_proba(test_feature)  # 获取0和1的概率分数，注意返回的是2列的数组
class_1_probs = probabilities[:,1]


random_forest_score = pd.DataFrame(class_1_probs)
data =pd.concat([data, random_forest_score], axis=1)
data.columns = [*data.columns[:-1], 'score']
data = data.sort_values(by=[data.columns[0],data.columns[-1], data.columns[9]], ascending=[True, False, False])
data.to_csv(result_path+'_candi_output.csv', index=False)



grouped = data.groupby(data.iloc[:, 0])
# 创建一个空的 DataFrame 来存放结果
result_df = pd.DataFrame(columns=data.columns)
# 遍历每个分组
for name, group in grouped:
    sorted_group = group.sort_values(by=['score', 'match fragment'], ascending=[False,False])
    max_row = sorted_group.iloc[0]
    result_df = result_df.append(max_row, ignore_index=True)

result_df = result_df.sort_values(by=[result_df.columns[-1], result_df.columns[9]], ascending=[False, False])


# 将 DataFrame 保存为 Excel 文件
result_df.to_csv(result_path+'_output.csv', index=False)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    2.3s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   14.9s
[Parallel(n_jobs=10)]: Done 268 tasks      | elapsed:   36.9s
[Parallel(n_jobs=10)]: Done 492 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  1.8min
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:  2.3min finished
